In [ ]:
from git_root import git_root
import os
from pathlib import Path
import numpy as np

os.chdir("/Users/kevinklein/Code/life-monitor")

from monitoring import cal, setup, utils, yearly

_FIRST_YEAR = 2022
_LAST_YEAR = 2024

import yaml

with open("env-vars.yaml", "r") as stream:
    try:
        env_vars = yaml.safe_load(stream)
        for key, value in env_vars.items():
            os.environ[key] = value
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
def _get_df(sport):
    service = setup.get_calendar_service()
    start = utils.first_of_jan_timestamp(year=_FIRST_YEAR)
    end = utils.last_of_dec_timestamp(year=_LAST_YEAR)
    events = cal.get_filtered_events(
        service,
        start,
        end,
        "summary",
        filter_value=sport,
    )
    y = cal.prune_events(events)

    df = cal.get_dataframe(events)
    return df

In [ ]:
    service = setup.get_calendar_service()
    start = utils.first_of_jan_timestamp(year=_FIRST_YEAR)
    end = utils.last_of_dec_timestamp(year=_LAST_YEAR)
    events = cal.get_filtered_events(
        service,
        start,
        end,
        "color",
        filter_value="4",
    )

In [ ]:
def _get_df_exercise():
    service = setup.get_calendar_service()
    start = utils.first_of_jan_timestamp(year=_FIRST_YEAR)
    end = utils.last_of_dec_timestamp(year=_LAST_YEAR)
    events = cal.get_filtered_events(
        service,
        start,
        end,
        "color",
        filter_value="4",
    )
    y = cal.prune_events(events)
    df = cal.get_dataframe(events)
    
    def is_endurance(title: str) -> bool:
        return title in ["running", "cross-country skiing", "cycling", "tennis", "hiking", "swimming", 
                         "snowshoe hiking", "inline skating", "elliptical", "padel", "elliptical"]
    
    def is_strength(title: str) -> bool:
        return title in ["gym", "climbing", "gym: ub", "gym: c", "gym: lb", "via ferrata"]

    df["endurance"] = df["title"].apply(is_endurance)
    df["strength"] = df["title"].apply(is_strength)
    return df

In [ ]:
import matplotlib.pyplot as plt

def _relative_frequency_plot(probabilities, fig, ax):
    disciplines = list(sorted(probabilities[2023].keys(), key=lambda d: probabilities[2023][d], reverse=True))
    n_disciplines = len(disciplines)
    for year in [2023, 2022]:
        values = [probabilities[year][discipline] for discipline in disciplines]
        ax.bar(range(n_disciplines), values, align='center', label=str(year), alpha=.3)
    ax.set_xticks(range(n_disciplines), disciplines, rotation="vertical")
    ax.legend()
    fig.tight_layout()

In [ ]:
dfs = {}
for sport in [utils.Sport.running, utils.Sport.cycling, utils.Sport.gym, utils.Sport.climbing]:
    dfs[sport] = _get_df(sport)

In [ ]:
dfs[utils.Sport.running]

In [ ]:
df_exercise = _get_df_exercise()

# Individual disciplines

### Climbing

In [ ]:
dfs[utils.Sport.climbing].groupby("year")["title"].count()

### Gym

In [ ]:
gym = dfs[utils.Sport.gym]

In [ ]:
gym[gym["year"] == 2023]["title"].value_counts()

In [ ]:
gym.groupby("year")["title"].count()

### Running

In [ ]:
dfs[utils.Sport.running].groupby("year")["distance"].sum()

In [ ]:
dfs[utils.Sport.running].groupby("year")["distance"].count()

In [ ]:
dfs[utils.Sport.running].groupby("year")["distance"].median()

In [ ]:
df_exercise.groupby("year")["title"].agg(lambda x: x[x == "running"].count() / x.count())

In [ ]:
cal.plot_cumulative_day_distances(dfs[utils.Sport.running], utils.Sport.running, Path(git_root()))

### Cycling

In [ ]:
dfs[utils.Sport.cycling].groupby("year")["distance"].sum()

In [ ]:
dfs[utils.Sport.cycling].groupby("year")["distance"].count()

In [ ]:
cal.plot_cumulative_day_distances(dfs[utils.Sport.cycling], utils.Sport.cycling, Path(git_root()))

# Overall exercise

In [ ]:
year_counts = df_exercise.groupby("year")["title"].count().to_dict()
year_counts

In [ ]:
df_exercise.groupby("year")["title"].nunique()

In [ ]:
probabilities, entropies = yearly.variety(df_exercise)

In [ ]:
fig_context, axs_context = plt.subplots(ncols=2, figsize=(10, 5))
_relative_frequency_plot(probabilities, fig_context, axs_context[0])
axs_context[0].set_title("Relative frequencies of various disciplines")

### Setup for radar plot

In [ ]:
endurances = {k: v / 365 for k, v in df_exercise.groupby("year")["endurance"].sum().to_dict().items()}
endurances

In [ ]:
strengths = {k: v / 365 for k,v in df_exercise.groupby("year")["strength"].sum().to_dict().items()}
strengths

In [ ]:
from math import log
max_entropy = log(len(probabilities[2022]), 2)
varieties = {k: v/max_entropy for k, v in entropies.items()}
varieties

In [ ]:
from collections import defaultdict
durations_h = df_exercise.index.diff().total_seconds() / 3600
df_exercise_aux = df_exercise.copy()
df_exercise_aux["durations"] = durations_h
consistencies = df_exercise_aux.groupby("year")["durations"].std().to_dict()
consistencies = {k: v/96 for k,v in consistencies.items()}
consistencies

In [ ]:
# Inverting the order to make sure color-coding checks out.
df_exercise_aux.iloc[::-1].groupby("year", sort=False)["durations"].hist(
    ax=axs_context[1], figure=fig_context, legend=True, bins=20, alpha=.3
)
axs_context[1].set_title("Durations [h] between two consecutive exercise activities")
axs_context[1].legend()

In [ ]:
fig_context.savefig("context.svg")
fig_context

In [ ]:
labels = {
    "Endurance": [0, .25, .5, .75, 1],
    "Variety": [0, round(max_entropy/4, 2), round(max_entropy/2, 2), round(max_entropy*3/4, 2), round(max_entropy,2)],
    "Strength": [0, .25, .5, .75, 1],
    "Consistency": [96, 62, 48, 24, 0],
}

In [ ]:
dimensions = ["Endurance",  "Variety", "Strength", "Consistency"]
n_dimensions = len(dimensions)
radar_years = [2024, 2023, 2022]
radar_data = {year: [endurances[year], varieties[year], strengths[year], (1-consistencies[year])] for year in radar_years}

In [ ]:
colors = ["b", "r", "g"]

### Radar plot

In [ ]:
fig = plt.figure()
radar = yearly.Radar(fig, dimensions, labels.values())
for year, color in zip(radar_years, colors):
    radar.plot(radar_data[year], '-', lw=2, color=color, alpha=0.4, label=year)
    radar.fill(radar_data[year], lw=2, color=color, alpha=.2)
radar.ax.legend()
radar.ax.tick_params(pad=40)
fig.savefig("radar.svg", bbox_inches='tight')